# Exercício de API

## Características

- Consulta de dados de NBA: https://rapidapi.com/theapiguy/api/free-nba
- Obter jogos disputados casa/fora por equipa
- Indica se equipa ganhou/perdeu jogo
- % de vitórias da equipa

## Problemas

- Api apresenta problemas na pesquisa de todos os jogos por id da equipa, por sesson
- A pesquisa de todos os jogos da equipa será feita apenas aos primeiros 100 jogos obtidos (api devolve no máximo 100 jogos por pedido)

## Resolução

- Obter todos os jogos da lista total (100 neste caso) e posteriormente filtrar através de código pela equipa pretendida (através de da paginação)

import requests
import json
import os

In [7]:
import requests
import json
import os

In [8]:
def get_all_teams():
    url = "https://free-nba.p.rapidapi.com/teams"
    querystring = {"page":"0"}
    headers = {
        'x-rapidapi-key': "99dc416392msh9e26edeab045e36p197c74jsn7009ee9adeef",
        'x-rapidapi-host': "free-nba.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers)#, params=querystring)
    #tratamos o json e devolvemos um dic com name - id
    return_dic = {}
    team_dic=json.loads(response.text)
    
    for t in team_dic['data']:
        return_dic[t['full_name']] = t['id']
    
    return return_dic


In [9]:
dic = get_all_teams()
print("Equipas Disponiveis:")
for k in dic.keys():
    print("\t{0}".format(k))
print('\n')


Equipas Disponiveis:
	Atlanta Hawks
	Boston Celtics
	Brooklyn Nets
	Charlotte Hornets
	Chicago Bulls
	Cleveland Cavaliers
	Dallas Mavericks
	Denver Nuggets
	Detroit Pistons
	Golden State Warriors
	Houston Rockets
	Indiana Pacers
	LA Clippers
	Los Angeles Lakers
	Memphis Grizzlies
	Miami Heat
	Milwaukee Bucks
	Minnesota Timberwolves
	New Orleans Pelicans
	New York Knicks
	Oklahoma City Thunder
	Orlando Magic
	Philadelphia 76ers
	Phoenix Suns
	Portland Trail Blazers
	Sacramento Kings
	San Antonio Spurs
	Toronto Raptors
	Utah Jazz
	Washington Wizards




In [10]:
def get_all_games_team(team_id):
    url = "https://free-nba.p.rapidapi.com/games"
    querystring = {"per_page":"100","page":"0"}
    headers = {
        'x-rapidapi-key': "acfd71aff7msh0594993c6052cc1p1faf6cjsn02dbd6af547a",
        'x-rapidapi-host': "free-nba.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    games_dic=json.loads(response.text)
    win = 0
    total = 0
    for g in games_dic['data']:
        if g['home_team']['id'] == team_id:
            total +=1
            if g['home_team_score'] > g['visitor_team_score']:
                win += 1
                print("WIN - {0} ({1} pts) VS {2} ({3} pts)".format(g['home_team']['full_name'],g['home_team_score'],g['visitor_team']['full_name'],g['visitor_team_score']))
            else:
                print("LOST - {0} ({1} pts) VS {2} ({3} pts)".format(g['home_team']['full_name'],g['home_team_score'],g['visitor_team']['full_name'],g['visitor_team_score']))
        elif g['visitor_team']['id'] == team_id:
            total +=1
            if g['home_team_score'] < g['visitor_team_score']:
                win+=1
                print("WIN - {0} ({1} pts) VS {2} ({3} pts)".format(g['home_team']['full_name'],g['home_team_score'],g['visitor_team']['full_name'],g['visitor_team_score']))
            else:
                print("LOST - {0} ({1} pst) VS {2} ({3} pts)".format(g['home_team']['full_name'],g['home_team_score'],g['visitor_team']['full_name'],g['visitor_team_score']))
                
    print("\n% WIN: {0:.2f}%".format(win/total*100))

In [11]:
def get_results_teams(team):
    dic = get_all_teams()
    id_team = dic.get(team)
    
    if id_team:
        get_all_games_team(id_team)
    else:
        print("Equipa indicada não existe.")

In [12]:
team_name = input("Indique o nome da equipa?\n")
get_results_teams(team_name)

WIN - Boston Celtics (126 pts) VS Charlotte Hornets (94 pts)
LOST - Boston Celtics (112 pts) VS LA Clippers (123 pts)
WIN - Philadelphia 76ers (109 pts) VS Boston Celtics (112 pts)
WIN - Boston Celtics (118 pts) VS Detroit Pistons (110 pts)
LOST - Milwaukee Bucks (98 pst) VS Boston Celtics (97 pts)
WIN - Golden State Warriors (95 pts) VS Boston Celtics (128 pts)
WIN - Boston Celtics (110 pts) VS Miami Heat (105 pts)

% WIN: 71.43%


# Exercício de Scrapping

## Características

- Obter produtos da primeira página do site do ebay 
- Procurar por nome do producto
- Resultado contém Nome, Estado, Preço e Moeda

## Problemas

- O valor de pesquisa pode resultar numa lista vazia (ex: samsung, mi band)
- Funciona com pesquisas mais detalhadas (ex: pixel 5, galaxy s7, iphone 7, ...)

## Notas

- Validado com o colega de grupo na resolução deste problema o código aprensta comportamento diferente em função da máquina onde corre
- Com mais tempo teria de ser feita analise mais detalha o porque de acontecer esta situação

In [1]:
import pandas as pd
import requests # pedidos http 
from bs4 import BeautifulSoup # web scraping

In [2]:

def get_data(url, parse):
    """
    returns a parsed html 
    """
    raw_html = requests.get(url, parse).content
    return BeautifulSoup(raw_html)


In [9]:
def list_ebay_products(html_result):
    #searches for items in the html and extracts information
    names=[]
    states=[]
    prices=[]
    currencies=[]
    for i in html_result:
        dissearch=i.find_all('div',class_='s-item__wrapper clearfix')
        for j in dissearch:
            name=j.find_all('h3',class_='s-item__title')
            state=j.find_all('span',class_='SECONDARY_INFO')
            price=j.find_all('span',class_='s-item__price')
            try:
                #if the entry is not empty, they are placed in an appropriate list
                names.append(name[0].get_text())
                states.append(state[0].get_text())
                value= price[0].get_text().split(' ')
                currencies.append(value[0])
                try:
                    prices.append(float(value[1].replace(',','.')))
                except:
                    prices.append(float(value[1].replace('\xa00','').replace(',','.')))
                    print("error")
            except IndexError:
                #if the entry is empty, the programa skips to the enxt entry
                pass
    return pd.DataFrame(list(zip(names,states,prices,currencies)),columns=[ 'Listing Name', 'State', 'Price', 'Currency'])

In [25]:

def search_product_ebay(product):
    
    url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw='+product+'&_sacat=0'

    raw_html = requests.get(url).content
    html = BeautifulSoup(raw_html)
    #html = get_data(url, "html.parsed")
    lista=html.find_all('li',class_='s-item')
    
    result = list_ebay_products(lista)
    return result

In [26]:
product=input('What do you want to search for?') #asks for input to search for

result_search = search_product_ebay(str(product))

result_search

What do you want to search for? galaxy s7


,Listing Name,State,Price,Currency
0,,Seminovo,85.95,EUR
1,Samsung Galaxy S7 SM-G930F 32GB Android Smartp...,Recondicionado,133.16,EUR
2,⭐⭐ Samsung Galaxy S7 (SM-G930F) 32GB Desbloque...,Recondicionado,121.59,EUR
3,SAMSUNG GALAXY S7 G930 G930F 32GB-Desbloqueado...,Seminovo,41.50,EUR
4,Samsung Galaxy S7 SM-G930F - 32GB - Smartphone...,Recondicionado,69.49,EUR
...,...,...,...,...
57,Google bloqueado Samsung Galaxy S7 Edge G935P ...,Seminovo,20.50,EUR
58,Novo anúncioSamsung Galaxy S7 SM-G930F - 32GB ...,Novo em folha,159.80,EUR
59,Samsung Galaxy S7 SM-G930A 5.1 Polegadas 4G LT...,Novo em folha,150.98,EUR
60,Galaxy S7 SM-G930F Original Unlocked Samsung 3...,Somente peças,40.31,EUR
